<a href="https://colab.research.google.com/github/JiHyun13/coding-academy-location-analysis/blob/main/%ED%95%99%EC%9B%90%EA%B5%90%EC%8A%B5%EC%86%8C%EC%A2%8C%ED%91%9C%EC%B1%84%EC%9B%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import requests
import time
import re
import folium
import geopandas as gpd
from shapely.geometry import Point

#### 2-1-1. 병원_최종 csv파일 만듦. (볼 필요X)

In [ ]:
df = pd.read_csv(r"C:\Users\zappe\Desktop\Test_for_python\데이터분석개론\team_proj\data\학원교습소정보.csv",encoding='utf-8')

In [ ]:
df.head() #데이터 구조 확인

,학원교습소명,학원명,분야명,교습계열명,교습과정목록명,교습과정명,도로명주소,y,x,도로명우편번호,좌표정보x(epsg5174),좌표정보y(epsg5174)
0,교습소,123에듀진학상담지도교습소,입시.검정 및 보습,진학지도,NaN,진학상담지도,"서울특별시 강남구 삼성로63길 19, 4층일부 (대치동)",37.499498,127.058597,6209,NaN,NaN
1,교습소,301아카데미(Academy)영어교습소,입시.검정 및 보습,보통교과,문법 영어,보습,"서울특별시 강남구 개포로82길 7, 3층 301호 (개포동, 삼성빌딩)",37.488777,127.067324,6329,NaN,NaN
2,교습소,3030영어교습소,입시.검정 및 보습,NaN,NaN,NaN,"서울특별시 강남구 광평로10길 15, 115호(일원동,상록수아파트단지내상가동)",37.482024,127.082132,6360,NaN,NaN
3,교습소,365국어교습소,입시.검정 및 보습,NaN,NaN,NaN,"서울특별시 강남구 삼성로 150, 158호, 165호(대치동,한보종합상가)",37.493393,127.067350,6288,NaN,NaN
4,교습소,9087스튜디오미술교습소,예능(대),예능(중),드로잉,미술,"서울특별시 서초구 바우뫼로20길 25, 201호 (양재동, 서두빌딩)",37.474134,127.034079,6755,NaN,NaN


In [ ]:
def get_coords_vworld(address, vworld_api_key):
    url = "https://api.vworld.kr/req/address"
    params = {
        "service": "address",
        "request": "getcoord",
        "version": "2.0",
        "crs": "EPSG:4326",
        "address": address,
        "format": "json",
        "type": "ROAD",
        "key": vworld_api_key
    }

    response = requests.get(url, params=params)
    if response.status_code == 200:
        result = response.json()
        try:
            x = result['response']['result']['point']['x']
            y = result['response']['result']['point']['y']
            return float(x), float(y)
        except:
            return None, None
    return None, None

def fill_missing_coords(df, api_key, delay=0.3):

    for idx, row in df.iterrows():

        full_address = row['도로명주소']
        if full_address is None or pd.isna(full_address):
            print(f"❌ [{idx}] 주소 없음")
            continue
        x, y = get_coords_vworld(full_address, api_key)

        if x and y:
            df.at[idx, 'x'] = x
            df.at[idx, 'y'] = y
            print(f"✅ [{idx}] 좌표 채움: {x}, {y}")
        else:
            print(f"❌ [{idx}] 좌표 못 찾음: {full_address}")

        time.sleep(delay)  # 요청 간 시간 간격 (초당 3~5회 이하 추천)

    return df


In [ ]:
vworld_key = "239AEE7D-C80A-374B-B39F-A2E7A556C870"

In [ ]:
df_null = fill_missing_coords(df, vworld_key)

✅ [0] 좌표 채움: 127.058598476, 37.499487556
✅ [1] 좌표 채움: 127.067318726, 37.488727546
✅ [2] 좌표 채움: 127.081271422, 37.481123286
✅ [3] 좌표 채움: 127.066632305, 37.494611767
✅ [4] 좌표 채움: 127.034106454, 37.474124852
✅ [5] 좌표 채움: 127.060364555, 37.483149078
✅ [6] 좌표 채움: 127.08565373, 37.492726031
✅ [7] 좌표 채움: 127.023765113, 37.524251594
✅ [8] 좌표 채움: 127.060014732, 37.501257365
✅ [9] 좌표 채움: 127.034947668, 37.52981192
✅ [10] 좌표 채움: 127.027063491, 37.523597039
✅ [11] 좌표 채움: 127.059112164, 37.501102272
✅ [12] 좌표 채움: 127.039321684, 37.526989294
✅ [13] 좌표 채움: 127.079650885, 37.482418946
✅ [14] 좌표 채움: 127.076520305, 37.482625984
✅ [15] 좌표 채움: 127.050310152, 37.500552823
✅ [16] 좌표 채움: 126.995679938, 37.503081407
✅ [17] 좌표 채움: 127.05303972, 37.499891549
✅ [18] 좌표 채움: 127.060376636, 37.493638098
✅ [19] 좌표 채움: 127.064872435, 37.500741349
✅ [20] 좌표 채움: 127.095365811, 37.471999473
✅ [21] 좌표 채움: 127.046378922, 37.498688354
✅ [22] 좌표 채움: 127.063391355, 37.495457209
✅ [23] 좌표 채움: 127.094973977, 37.47191305
✅ [24]

In [ ]:
#좌표가 비어있으면 해당 행 삭제
df_final = df_null.dropna(subset=['x', 'y'])

In [ ]:
df_final.to_csv("최종학원.csv", index=False, encoding='utf-8-sig')

: 